In [ ]:
from flask import Flask, request, render_template
import torch
import torch.nn as nn
from konlpy.tag import Okt
import pickle

# Flask 애플리케이션 초기화
app = Flask(__name__)

# 모델 파일 경로 및 로드
MODEL_PATH = "/mnt/data/best_lstm_model.pth"
model = torch.load(MODEL_PATH)
model.eval()

# 한국어 토크나이저 초기화
tokenizer = Okt()

# 예측 함수 정의
def predict_episode(comment):
    # 토큰화
    tokens = tokenizer.morphs(comment)
    
    # 여기에 적절한 인덱스 매핑과 패딩이 필요합니다.
    # 예를 들어, vocab이 있다면 vocab로 인덱스로 바꾸고 padding을 추가합니다.
    # 다음과 같은 텐서 생성이 필요합니다 (예: `torch.tensor([[1, 2, 3]])` 형태)
    text_tensor = torch.tensor([1, 2, 3])  # 예시
    
    # 모델을 통해 예측
    with torch.no_grad():
        output = model(text_tensor)
        predicted = torch.argmax(output, dim=1).item()
        
    return predicted

# 라우트 정의
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        comment = request.form['comment']
        episode = predict_episode(comment)
        return render_template('index.html', comment=comment, episode=episode)
    return render_template('index.html')

# HTML 템플릿 (index.html)
@app.route('/templates/index.html')
def render_template():
    return '''
    <!doctype html>
    <html lang="en">
      <head>
        <meta charset="utf-8">
        <title>무한도전 회차 예측</title>
      </head>
      <body>
        <h1>무한도전 댓글 기반 회차 예측</h1>
        <form method="post" action="/">
          <label for="comment">댓글을 입력하세요:</label><br>
          <input type="text" id="comment" name="comment" required><br><br>
          <input type="submit" value="예측">
        </form>
        {% if episode %}
          <h2>예측된 회차: {{ episode }}</h2>
          <p>댓글: {{ comment }}</p>
        {% endif %}
      </body>
    </html>
    '''

# Flask 앱 실행
if __name__ == '__main__':
    app.run(debug=True)

